In [1]:
# Imports
import imageio
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
from tqdm.notebook import tqdm

In [2]:
# Configure plotting style
plt.rcParams['figure.figsize'] = [10, 1]

In [3]:
# Constants
posCoord   = (50, 60) # 'position' starting coordinates
posChars   = 9*3 + 2  # number of 'position' chars
angleCoord = (50, 74) # 'angle' starting coordinates
angleChars = 6*2 + 1  # number of 'angle' chars (ignoring 3rd angle)
velCoord   = (50, 88) # 'velocity' starting coordinates
velChars   = 7        # number of 'velocity' chars

In [4]:
# Training videos
trainingVideos = [
    'Gameplay/Noise1_Trim.mp4',
    'Gameplay/Noise2_Trim.mp4',
    'Gameplay/Noise3_Trim.mp4'
]

In [5]:
# Extract char image from video frame
def getCharImage(frame, pos, N):
    # Char properties
    charWidth  = 8  # character width
    charHeight = 12 # character height
    
    # Calculate coordinates
    xStart = pos[0] + charWidth*N
    xStop  = xStart + charWidth
    yStart = pos[1]
    yStop  = yStart + charHeight
    
    # Extract image
    return frame[yStart:yStop, xStart:xStop]

In [6]:
# Load in training data if it exists
if os.path.exists('trainingData.db'):
    # Load training data
    data   = pickle.load(open('trainingData.db', 'rb'))
    images = data['images']
    labels = data['labels']
else:
    # Iterate through each training video
    images = []
    for videoLoc in tqdm(trainingVideos, 'Parsing Training Videos', leave=False):
        # Import video
        vid = imageio.get_reader(videoLoc, 'ffmpeg')

        # Iterate through video frames
        # Only use 3 frames per second to ensure variety in training data
        nFrames  = vid.count_frames()
        frameIdx = np.arange(0, nFrames, 10)
        for idx in tqdm(frameIdx, 'Parsing Training Frames', leave=False):
            # Get frame
            frame = vid.get_data(idx)

            # Calculate mean frame
            frameMean = np.mean(frame, 2)

            # Extract angle data
            for i in range(angleChars):
                images.append(getCharImage(frameMean, angleCoord, i))

    # Save training data
    labels = []
    data = {
        'images': images,
        'labels': labels
    }
    pickle.dump(data, open('trainingData.db', 'wb'))

In [69]:
# Label training data
groupSize = 30
for i in np.arange(len(labels), len(images), groupSize):
    # Display image
    if i + groupSize > len(images):
        nImages = len(images) - i
        if nImages == 1:
            plt.imshow(images[i])
        else:
            plt.imshow(np.concatenate(images[i:i + nImages], 1))
    else:
        nImages = groupSize
        plt.imshow(np.concatenate(images[i:i + groupSize], 1))
    plt.show()
    
    # Collect and store label
    done = False
    while not done:
        labelsInput = [char for char in input()]
        done = len(labelsInput) == nImages
    labels += labelsInput

In [71]:
# Save training data
data = {
    'images': images,
    'labels': labels
}
pickle.dump(data, open('trainingData.db', 'wb'))